<a href="https://colab.research.google.com/github/Leo777Deo/CFD_Project/blob/main/BumpConv/Different_cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# MARTINA SENZA TRANS
from os import write
import numpy as np
import matplotlib.pyplot as plt
import math

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 0 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.02 # Additive quantity to the end point of the convergent to close it
thick_conv2 = 0.2 * thick_conv
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

x_start_up, y_start_up = x_vals[index_line_bump], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

x_start_conv = x_vals[index_line_bump - 1]
y_start_conv = y_start_up - (x_start - x_start_conv)*np.tan(angle_rad)


# Define the length of the line segment
End_line_param = 4
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up + line_length * np.sin(angle_rad)

# Define Far Field
Y_DIST = 50 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -3 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = x_end


size_bl = 0.00001
n_bl = 65 # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1

# .geo file creation:
with open("Bumpconv.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.01;\n") # Size of cells close to bump and convergent
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

    # Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

    # Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")
    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1
    upper_conv = upper_point_end + 1
    lower_conv = upper_conv + 1
    circle_centre = lower_conv + 1
    center_convergent = circle_centre + 1

    # Write the convergent shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")
    file.write(f"Line({3}) = {{{new_point_start}, {new_point_end}}};\n")

    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")
    file.write(f"Line({4}) = {{{upper_point_end}, {upper_point_start}}};\n")

    y_upper_conv = y_start_conv - thick_conv2
    y_lower_conv = y_start_conv + thick_conv2 - thick_conv

    file.write(f"Point({upper_conv}) = {{{x_start_conv}, {y_upper_conv}, 0, h}};\n")
    file.write(f"Point({lower_conv}) = {{{x_start_conv}, {y_lower_conv}, 0, h}};\n")
    file.write(f"Point({circle_centre}) = {{{x_start_conv + (x_start_up - x_start_conv)/200}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Computation of the radius of the circle of the convergent intake, given the center position and two points
    hhh = x_start_conv + (x_start_up - x_start_conv)/200      #float(input("Inserisci la coordinata x del centro (h): "))
    kkk = (y_upper_conv + y_lower_conv)/2      #float(input("Inserisci la coordinata y del centro (k): "))
    xxx1 = x_start_conv     #float(input("Inserisci la coordinata x del punto sulla circonferenza: "))
    yyy1 = y_upper_conv     #float(input("Inserisci la coordinata y del punto sulla circonferenza: "))
    raggio1 = math.sqrt((xxx1 - hhh)**2 + (yyy1 - kkk)**2)
    front_nose = raggio1

    file.write(f"Point({center_convergent}) = {{{x_start_conv + (x_start_up - x_start_conv)/200 - front_nose}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Define connecting lines for closed convergent surface
    file.write(f"Line({5}) = {{{lower_conv}, {new_point_start}}};\n")
    file.write(f"Line({6}) = {{{upper_point_start}, {upper_conv}}};\n")
    file.write(f"Circle({16}) = {{{upper_conv}, {circle_centre}, {center_convergent}}};\n")  # Circle definition (start point, centre point, end point)
    file.write(f"Circle({19}) = {{{center_convergent}, {circle_centre}, {lower_conv}}};\n")  # Circle definition (start point, centre point, end point)


    # Boundary points for the entire domain
    boundary_point_1 = center_convergent + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1
    boundary_point_7 = boundary_point_6 + 1
    boundary_point_8 = boundary_point_7 + 1
    boundary_point_9 = boundary_point_8 + 1
    boundary_point_10 = boundary_point_9 + 1

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, 0, 0, H/4}};\n")

    file.write(f"Point({boundary_point_5}) = {{{X_START}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_6}) = {{{X_END}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {y_end - thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_8}) = {{{X_END}, {y_end_up + thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_10}) = {{{0}, {thick_bl}, 0, h}};\n")

    # Points for the B.L. of the convergent intake:
    # BL_conv_1 = boundary_point_10 + 1
    # BL_conv_2 = BL_conv_1 + 1
    # BL_conv_3 = BL_conv_2 + 1
    # BL_conv_4 = BL_conv_3 + 1
    # BL_conv_5 = BL_conv_4 + 1
    # BL_conv_6 = BL_conv_5 + 1
    # BL_conv_7 = BL_conv_6 + 1

    # file.write(f"Point({BL_conv_1}) = {{{x_end_up}, {y_end_up + thick_bl}, 0, h}};\n")
    # file.write(f"Point({BL_conv_2}) = {{{x_start_up}, {y_start_up + thick_bl}, 0, h}};\n")
    # file.write(f"Point({BL_conv_3}) = {{{x_start_conv}, {y_upper_conv + thick_bl}, 0, h}};\n")
    # file.write(f"Point({BL_conv_4}) = {{{x_start_conv}, {y_lower_conv - thick_bl}, 0, h}};\n")
    # file.write(f"Point({BL_conv_5}) = {{{x_start}, {y_start - thick_bl}, 0, h}};\n")
    # file.write(f"Point({BL_conv_6}) = {{{x_end}, {y_end - thick_bl}, 0, h}};\n")

    # Computation of the radius of the circle of the convergent intake B.L., given the center position and two points
    hhh2 = x_start_conv + (x_start_up - x_start_conv)/200      #float(input("Inserisci la coordinata x del centro (h): "))
    kkk2 = (y_upper_conv + y_lower_conv)/2      #float(input("Inserisci la coordinata y del centro (k): "))
    xxx2 = x_start_conv     #float(input("Inserisci la coordinata x del punto sulla circonferenza: "))
    yyy2 = y_upper_conv + thick_bl    #float(input("Inserisci la coordinata y del punto sulla circonferenza: "))
    raggio2 = math.sqrt((xxx2 - hhh2)**2 + (yyy2 - kkk2)**2)
    front_nose_BL = raggio2

    # file.write(f"Point({BL_conv_7}) = {{{x_start_conv + (x_start_up - x_start_conv)/200 - front_nose_BL}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Define lines for the main boundary
    file.write(f"Line({7}) = {{{boundary_point_1}, {boundary_point_6}}};\n")
    file.write(f"Line({8}) = {{{boundary_point_6}, {boundary_point_7}}};\n")
    file.write(f"Line({17}) = {{{boundary_point_7}, {new_point_end}}};\n")

    #file.write(f"Line({9}) = {{{new_point_end}, {upper_point_end}}};\n")

    file.write(f"Line({10}) = {{{upper_point_end}, {boundary_point_8}}};\n")
    file.write(f"Line({18}) = {{{boundary_point_8}, {boundary_point_2}}};\n")
    file.write(f"Line({11}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({12}) = {{{boundary_point_3}, {boundary_point_5}}};\n")
    file.write(f"Line({13}) = {{{boundary_point_5}, {boundary_point_4}}};\n")
    file.write(f"Line({14}) = {{{boundary_point_4},{1} }};\n")  # Connect back to start of bump
    file.write(f"Line({15}) = {{{new_point_start - 1},{boundary_point_1} }};\n")
    file.write(f"Line({20}) = {{{1},{boundary_point_10}}};\n")
    file.write(f"Line({21}) = {{{boundary_point_10},{boundary_point_5}}};\n")


    file.write(f"Transfinite Line{{{7}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{13}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{17}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{10}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{8}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{19}}} = {10} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{15}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{16}}} = {10} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{18}}} = {150} Using Progression {r};\n")

    file.write(f"Transfinite Line{{{14}}} = {300} Using Progression {1};\n")
    #file.write(f"Transfinite Line{{{19}}} = {150} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{20}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{21}}} = {300} Using Progression {1};\n")

    file.write("Line Loop(1) = {1, 2, 15, 7, 8, 17, -3, -5, -19, -16, -6, -4, 10, 18, 11, 12, -21, -20};\n")
    file.write("Line Loop(2) = {14,20,21,13};\n")


    #Define the fluid domain
    file.write("Plane Surface(1) = {1};\n") # It tells GMSH to mesh only on 1 and not account for 2
    file.write("Plane Surface(2) = {2};\n")
    file.write("Transfinite Surface{2} = {")
    file.write(f"{boundary_point_5},{boundary_point_4},{1},{boundary_point_10}")
    file.write("};\n")
    file.write("Recombine Surface{2};\n")


    # Boundary Layer:

    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {1};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[1].FanPointsList = {1};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")

    file.write("Field[2] = BoundaryLayer;\n")
    file.write("Field[2].CurvesList = {2,15};\n")
    file.write("Field[2].Quads = 1;\n")
    file.write(f"Field[2].Ratio = {r};\n")
    file.write(f"Field[2].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[2].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[2].FanPointsList = {")
    file.write(f"{34}")
    file.write("};\n")
    file.write("Field[2].FanPointsSizesList = {")
    file.write(f"{5}")
    file.write("};\n")
    file.write("BoundaryLayer Field = 2;\n")

    file.write("Field[3] = BoundaryLayer;\n")
    file.write("Field[3].CurvesList = {19,5,3};\n")
    file.write("Field[3].Quads = 1;\n")
    file.write(f"Field[3].Ratio = {r};\n")
    file.write(f"Field[3].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[3].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[3].FanPointsList = {")
    file.write(f"{58}")
    file.write("};\n")
    file.write("Field[3].FanPointsSizesList = {")
    file.write(f"{5}")
    file.write("};\n")
    file.write("BoundaryLayer Field = 3;\n")

    file.write("Field[4] = BoundaryLayer;\n")
    file.write("Field[4].CurvesList = {16,6,4};\n")
    file.write("Field[4].Quads = 1;\n")
    file.write(f"Field[4].Ratio = {r};\n")
    file.write(f"Field[4].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[4].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[4].FanPointsList = {")
    file.write(f"{55}")
    file.write("};\n")
    file.write("Field[4].FanPointsSizesList = {")
    file.write(f"{5}")
    file.write("};\n")
    file.write("BoundaryLayer Field = 4;\n")


    # Physical Surface definition

    #file.write("Line Loop(20) = {14, 1, 2, 15, 7, 8, -17, -3, -5, -19, -16, -6, -4, 10, 18, 11, 12, 13};\n")
    #file.write("Plane Surface(10) = {20};\n")
    file.write("Physical Surface('VOLUME') = {1:2};\n")
    #file.write("Physical Surface('VOLUME2') = {2};\n")
    #file.write("Physical Surface('VOLUME3') = {4};\n")
    #file.write("Physical Surface('VOLUME4') = {5};\n")
    #file.write("Physical Surface('VOLUME5') = {6};\n")
    #file.write("Physical Surface('VOLUME6') = {7};\n")
    #file.write("Physical Surface('VOLUME7') = {8};\n")
    #file.write("Physical Surface('VOLUME8') = {9};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{12},{13}}};\n")

    file.write("Physical Line('lower_wall') = {")
    file.write(f"{14},")
    file.write("1, 2")
    file.write(f",{15}")
    file.write("};\n")

    #file.write(f"Physical Line('upper wall') = {{{11}}};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{3}, {4}, {6}, {16}, {19}, {5}")
    file.write("};\n")
    file.write(f"Physical Line('outlet') = {{{10}, {18},{11}}};\n")
    file.write(f"Physical Line('ENGINE') = {{{7}, {8}, {17}}};\n")



from google.colab import files
files.download("Bumpconv.geo")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# RUNNA BENE, MA HA IL OUTLET LONTANO, NO SENSE

from os import write
import numpy as np
import matplotlib.pyplot as plt
import math

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 0 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.02 # Additive quantity to the end point of the convergent to close it
thick_conv2 = 0.2 * thick_conv
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

x_start_up, y_start_up = x_vals[index_line_bump], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

x_start_conv = x_vals[index_line_bump - 1]
y_start_conv = y_start_up - (x_start - x_start_conv)*np.tan(angle_rad)


# Define the length of the line segment
End_line_param = 4
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up + line_length * np.sin(angle_rad)

# Define Far Field
Y_DIST = 25 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST_START = -1 # Multiplication factor of START x point of bump to create far field X
X_DIST_END = 1 # Multiplication factor of END x point of bump to create far field X
X_START = X_DIST_START * x_vals[np.argmax(x_vals)]
X_END = X_DIST_END * x_vals[np.argmax(x_vals)]
#X_END=x_end



size_bl = 0.00001
n_bl = 65 # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1

# .geo file creation:
with open("mesh.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.1;\n") # Size of cells close to bump and convergent
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

    # Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

    # Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")
    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1
    upper_conv = upper_point_end + 1
    lower_conv = upper_conv + 1
    circle_centre = lower_conv + 1
    center_convergent = circle_centre + 1

    # Write the convergent shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")
    file.write(f"Line({3}) = {{{new_point_start}, {new_point_end}}};\n")

    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")
    file.write(f"Line({4}) = {{{upper_point_end}, {upper_point_start}}};\n")

    y_upper_conv = y_start_conv - thick_conv2
    y_lower_conv = y_start_conv + thick_conv2 - thick_conv

    file.write(f"Point({upper_conv}) = {{{x_start_conv}, {y_upper_conv}, 0, h}};\n")
    file.write(f"Point({lower_conv}) = {{{x_start_conv}, {y_lower_conv}, 0, h}};\n")
    file.write(f"Point({circle_centre}) = {{{x_start_conv + (x_start_up - x_start_conv)/200}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Computation of the radius of the circle of the convergent intake, given the center position and two points
    hhh = x_start_conv + (x_start_up - x_start_conv)/200      #float(input("Inserisci la coordinata x del centro (h): "))
    kkk = (y_upper_conv + y_lower_conv)/2      #float(input("Inserisci la coordinata y del centro (k): "))
    xxx1 = x_start_conv     #float(input("Inserisci la coordinata x del punto sulla circonferenza: "))
    yyy1 = y_upper_conv     #float(input("Inserisci la coordinata y del punto sulla circonferenza: "))
    raggio1 = math.sqrt((xxx1 - hhh)**2 + (yyy1 - kkk)**2)
    front_nose = raggio1

    file.write(f"Point({center_convergent}) = {{{x_start_conv + (x_start_up - x_start_conv)/200 - front_nose}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Define connecting lines for closed convergent surface
    file.write(f"Line({5}) = {{{lower_conv}, {new_point_start}}};\n")
    file.write(f"Line({6}) = {{{upper_point_start}, {upper_conv}}};\n")
    file.write(f"Circle({16}) = {{{upper_conv}, {circle_centre}, {center_convergent}}};\n")  # Circle definition (start point, centre point, end point)
    file.write(f"Circle({19}) = {{{center_convergent}, {circle_centre}, {lower_conv}}};\n")  # Circle definition (start point, centre point, end point)


    # Boundary points for the entire domain
    boundary_point_1 = center_convergent + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1
    boundary_point_7 = boundary_point_6 + 1
    boundary_point_8 = boundary_point_7 + 1
    boundary_point_9 = boundary_point_8 + 1
    boundary_point_10 = boundary_point_9 + 1

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, 0, 0, H/4}};\n")

    file.write(f"Point({boundary_point_5}) = {{{X_START}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_6}) = {{{X_END}, {0}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_8}) = {{{x_end}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_10}) = {{{0}, {thick_bl}, 0, h}};\n")

    # Points for the B.L. of the convergent intake:
    BL_conv_1 = boundary_point_10 + 1
    BL_conv_2 = BL_conv_1 + 1
    BL_conv_3 = BL_conv_2 + 1
    BL_conv_4 = BL_conv_3 + 1
    BL_conv_5 = BL_conv_4 + 1
    BL_conv_6 = BL_conv_5 + 1
    BL_conv_7 = BL_conv_6 + 1

    file.write(f"Point({BL_conv_1}) = {{{x_end_up}, {y_end_up + thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_2}) = {{{x_start_up}, {y_start_up + thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_3}) = {{{x_start_conv}, {y_upper_conv + thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_4}) = {{{x_start_conv}, {y_lower_conv - thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_5}) = {{{x_start}, {y_start - thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_6}) = {{{x_end}, {y_end - thick_bl}, 0, h}};\n")

    # Computation of the radius of the circle of the convergent intake B.L., given the center position and two points
    hhh2 = x_start_conv + (x_start_up - x_start_conv)/200      #float(input("Inserisci la coordinata x del centro (h): "))
    kkk2 = (y_upper_conv + y_lower_conv)/2      #float(input("Inserisci la coordinata y del centro (k): "))
    xxx2 = x_start_conv     #float(input("Inserisci la coordinata x del punto sulla circonferenza: "))
    yyy2 = y_upper_conv + thick_bl    #float(input("Inserisci la coordinata y del punto sulla circonferenza: "))
    raggio2 = math.sqrt((xxx2 - hhh2)**2 + (yyy2 - kkk2)**2)
    front_nose_BL = raggio2

    file.write(f"Point({BL_conv_7}) = {{{x_start_conv + (x_start_up - x_start_conv)/200 - front_nose_BL}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Define lines for the main boundary
    file.write(f"Line({7}) = {{{boundary_point_1}, {boundary_point_6}}};\n")
    file.write(f"Line({8}) = {{{boundary_point_6}, {boundary_point_7}}};\n")
    file.write(f"Line({10}) = {{{boundary_point_8}, {boundary_point_7}}};\n")
    file.write(f"Line({9}) = {{{new_point_end}, {upper_point_end}}};\n")
    file.write(f"Line({17}) = {{{boundary_point_1}, {boundary_point_8}}};\n")
    file.write(f"Line({18}) = {{{boundary_point_7}, {boundary_point_2}}};\n")
    file.write(f"Line({11}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({12}) = {{{boundary_point_3}, {boundary_point_5}}};\n")
    file.write(f"Line({13}) = {{{boundary_point_5}, {boundary_point_4}}};\n")
    file.write(f"Line({14}) = {{{boundary_point_4},{1} }};\n")  # Connect back to start of bump
    file.write(f"Line({15}) = {{{new_point_start - 1},{boundary_point_1} }};\n")
    file.write(f"Line({20}) = {{{1},{boundary_point_10}}};\n")
    file.write(f"Line({21}) = {{{boundary_point_10},{boundary_point_5}}};\n")


    file.write(f"Transfinite Line{{{7}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{13}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{17}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{10}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{8}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{19}}} = {10} Using Progression {1};\n")
    #file.write(f"Transfinite Line{{{len(x_vals) + 10}}} = {150} Using Progression {1/r};\n")
    #file.write(f"Transfinite Line{{{len(x_vals) + 12}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{15}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{16}}} = {10} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{18}}} = {150} Using Progression {r};\n")

    file.write(f"Transfinite Line{{{14}}} = {300} Using Progression {1};\n")
    #file.write(f"Transfinite Line{{{19}}} = {150} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{20}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{21}}} = {300} Using Progression {1};\n")


    # Define lines for the B.L. in the convergent intake
    file.write(f"Line({30}) = {{{BL_conv_1}, {BL_conv_2}}};\n")
    file.write(f"Line({31}) = {{{BL_conv_2}, {BL_conv_3}}};\n")
    file.write(f"Circle({32}) = {{{BL_conv_3}, {circle_centre}, {BL_conv_7}}};\n")
    file.write(f"Line({33}) = {{{BL_conv_4}, {BL_conv_5}}};\n")
    file.write(f"Line({34}) = {{{BL_conv_5}, {BL_conv_6}}};\n")
    file.write(f"Line({35}) = {{{upper_point_end}, {BL_conv_1}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({36}) = {{{BL_conv_2}, {upper_point_start}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({37}) = {{{BL_conv_3}, {upper_conv}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({38}) = {{{lower_conv}, {BL_conv_4}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({39}) = {{{BL_conv_5}, {new_point_start}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({40}) = {{{BL_conv_6}, {new_point_end}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Circle({41}) = {{{BL_conv_7}, {circle_centre}, {BL_conv_4}}};\n")
    file.write(f"Line({42}) = {{{BL_conv_7}, {center_convergent}}};\n")  # vertical line in the B.L. extremis

    # Define Transfinite lines for the B.L. of the convergent intake
    file.write(f" Transfinite Line{{{3}, {4}, {30}, {34}}} = {400};\n")
    file.write(f" Transfinite Line{{{5}, {6}, {31}, {33}}} = {150};\n")
    file.write(f" Transfinite Line{{{16}, {19}, {32}, {41}}} = {50};\n")
    file.write(f" Transfinite Line{{{36}, {37}, {39}, {40}, {42}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f" Transfinite Line{{{35}, {38}}} = {n_bl} Using Progression {r};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    #file.write("Line Loop(1) = {")
    #file.write("1, 2,") # Spline 1 and 2
    #file.write(f"{15}, {7}, {8}, {17} ,{9}, {10}, {18}, {11}, {12}, {13}, {14},{19}")
    #file.write("};\n")

    #file.write("Line Loop(1) = {1, 2, 15, 7, 8, 34, 33, 41, 32, 31, 30, 18, 11, 12, -21, -20};\n")
    file.write("Line Loop(1) = {1, 2, 15, 17, 10, 18, 11, 12, -21, -20};\n")


    #file.write("Line Loop(2) = {4, 6, 16, 19, 5, 3, 9};\n")

    file.write("Line Loop(2) = {3,9,4,6,16,19,5};\n") #convergent

    # Define 5 loops fo the B.L. of the convergent intake
    file.write("Line Loop(3) = { 35, 30, 36, -4 };\n")
    file.write("Line Loop(4) = { -36, 31, 37, -6 };\n")
    file.write("Line Loop(5) = { 37, 16, -42, -32};\n")  # on the upper circular portion (nose)
    file.write("Line Loop(6) = { 42, 19, 38, -41};\n")  # on the lower circular portion (nose)
    file.write("Line Loop(7) = { 38, 33, 39, -5 };\n")
    file.write("Line Loop(8) = { -39, 34, 40, -3 };\n")

    file.write("Line Loop(9) = { 17,10,-8,-7 };\n") # lower right transf
    file.write("Line Loop(10) = {14,20,21,13};\n")# lower left transf

    # Define a Line Loop for all the convergent inlet ( comprehensive of th B.L.)
    # file.write("Line Loop(10) = {34, 33, 41, 32, 31, 30, 10, 9, 17};\n")
    #file.write("Plane Surface(10) = {10,2};\n")

    #Define the fluid domain
    file.write("Plane Surface(2) = {1,2,3,4,5,6,7,8};\n") # It tells GMSH to mesh only on 1 and not account for 2

    # Define 5 surfaces for B.L. of the convergent intake
    file.write("Plane Surface(3) = {3};\n")
    file.write("Plane Surface(4) = {4};\n")
    file.write("Plane Surface(5) = {5};\n")
    file.write("Plane Surface(6) = {6};\n")
    file.write("Plane Surface(7) = {7};\n")
    file.write("Plane Surface(8) = {8};\n")
    file.write("Plane Surface(9) = {9};\n")
    file.write("Plane Surface(10) = {10};\n")
    # Define the transfinite surfaces for the B.L. of the convergent intake
    file.write("Transfinite Surface{3};\n")
    file.write("Transfinite Surface{4};\n")
    file.write("Transfinite Surface{5};\n")
    file.write("Transfinite Surface{6};\n")
    file.write("Transfinite Surface{7};\n")
    file.write("Transfinite Surface{8};\n")
    file.write("Transfinite Surface{9};\n")
    file.write("Transfinite Surface{10};\n")
    file.write("Recombine Surface{3};\n")
    file.write("Recombine Surface{4};\n")
    file.write("Recombine Surface{5};\n")
    file.write("Recombine Surface{6};\n")
    file.write("Recombine Surface{7};\n")
    file.write("Recombine Surface{8};\n")
    file.write("Recombine Surface{9};\n")
    file.write("Recombine Surface{10};\n")

    # Boundary Layer:

    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {1};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[1].FanPointsList = {1};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")

    file.write("Field[2] = BoundaryLayer;\n")
    file.write("Field[2].CurvesList = {2,15};\n")
    file.write("Field[2].Quads = 1;\n")
    file.write(f"Field[2].Ratio = {r};\n")
    file.write(f"Field[2].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[2].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[2].FanPointsList = {")
    file.write(f"{34}")
    file.write("};\n")
    file.write("Field[2].FanPointsSizesList = {")
    file.write(f"{5}")
    file.write("};\n")
    file.write("BoundaryLayer Field = 2;\n")


    # Physical Surface definition

    file.write("Physical Surface('VOLUME') = {2:10};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{12},{13}}};\n")

    file.write("Physical Line('lower_wall') = {")
    file.write(f"{14},")
    file.write("1, 2")
    file.write(f",{15},7")
    file.write("};\n")

    #file.write(f"Physical Line('upper wall') = {{{11}}};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{3}, {4}, {6}, {16}, {19}, {5}")
    file.write("};\n")
    file.write(f"Physical Line('outlet') = {{{8}, {18},{11}}};\n")
    #file.write(f"Physical Line('ENGINE') = {{{7}, {8}, {17}}};\n")



from google.colab import files


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

68


In [ ]:
# NO TRANSFINITE:

from os import write
import numpy as np
import matplotlib.pyplot as plt
import math

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 0 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.002 # Additive quantity to the end point of the convergent to close it
thick_conv2 = 0.002 * thick_conv
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

x_start_up, y_start_up = x_vals[index_line_bump], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

x_start_conv = x_vals[index_line_bump - 1]
y_start_conv = y_start_up - (x_start - x_start_conv)*np.tan(angle_rad)


# Define the length of the line segment
End_line_param = 4
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up + line_length * np.sin(angle_rad)

# Define Far Field
Y_DIST = 50 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -3 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = x_end


size_bl = 0.00001
n_bl = 65 # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1

# .geo file creation:
with open("Bumpconv.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.01;\n") # Size of cells close to bump and convergent
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

    # Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

    # Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")
    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1
    upper_conv = upper_point_end + 1
    lower_conv = upper_conv + 1
    circle_centre = lower_conv + 1
    center_convergent = circle_centre + 1

    # Write the convergent shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")
    file.write(f"Line({3}) = {{{new_point_start}, {new_point_end}}};\n")

    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")
    file.write(f"Line({4}) = {{{upper_point_end}, {upper_point_start}}};\n")

    y_upper_conv = y_start_conv - thick_conv2
    y_lower_conv = y_start_conv + thick_conv2 - thick_conv

    file.write(f"Point({upper_conv}) = {{{x_start_conv}, {y_upper_conv}, 0, h}};\n")
    file.write(f"Point({lower_conv}) = {{{x_start_conv}, {y_lower_conv}, 0, h}};\n")
    file.write(f"Point({circle_centre}) = {{{x_start_conv + (x_start_up - x_start_conv)/200}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Computation of the radius of the circle of the convergent intake, given the center position and two points
    hhh = x_start_conv + (x_start_up - x_start_conv)/200      #float(input("Inserisci la coordinata x del centro (h): "))
    kkk = (y_upper_conv + y_lower_conv)/2      #float(input("Inserisci la coordinata y del centro (k): "))
    xxx1 = x_start_conv     #float(input("Inserisci la coordinata x del punto sulla circonferenza: "))
    yyy1 = y_upper_conv     #float(input("Inserisci la coordinata y del punto sulla circonferenza: "))
    raggio1 = math.sqrt((xxx1 - hhh)**2 + (yyy1 - kkk)**2)
    front_nose = raggio1

    file.write(f"Point({center_convergent}) = {{{x_start_conv + (x_start_up - x_start_conv)/200 - front_nose}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Define connecting lines for closed convergent surface
    file.write(f"Line({5}) = {{{lower_conv}, {new_point_start}}};\n")
    file.write(f"Line({6}) = {{{upper_point_start}, {upper_conv}}};\n")
    file.write(f"Circle({16}) = {{{upper_conv}, {circle_centre}, {center_convergent}}};\n")  # Circle definition (start point, centre point, end point)
    file.write(f"Circle({19}) = {{{center_convergent}, {circle_centre}, {lower_conv}}};\n")  # Circle definition (start point, centre point, end point)


    # Boundary points for the entire domain
    boundary_point_1 = center_convergent + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1
    boundary_point_7 = boundary_point_6 + 1
    boundary_point_8 = boundary_point_7 + 1
    boundary_point_9 = boundary_point_8 + 1
    boundary_point_10 = boundary_point_9 + 1

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, 0, 0, H/4}};\n")

    file.write(f"Point({boundary_point_5}) = {{{X_START}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_6}) = {{{X_END}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {y_end - thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_8}) = {{{X_END}, {y_end_up + thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_10}) = {{{0}, {thick_bl}, 0, h}};\n")

    # Points for the B.L. of the convergent intake:
    BL_conv_1 = boundary_point_10 + 1
    BL_conv_2 = BL_conv_1 + 1
    BL_conv_3 = BL_conv_2 + 1
    BL_conv_4 = BL_conv_3 + 1
    BL_conv_5 = BL_conv_4 + 1
    BL_conv_6 = BL_conv_5 + 1
    BL_conv_7 = BL_conv_6 + 1

    file.write(f"Point({BL_conv_1}) = {{{x_end_up}, {y_end_up + thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_2}) = {{{x_start_up}, {y_start_up + thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_3}) = {{{x_start_conv}, {y_upper_conv + thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_4}) = {{{x_start_conv}, {y_lower_conv - thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_5}) = {{{x_start}, {y_start - thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_6}) = {{{x_end}, {y_end - thick_bl}, 0, h}};\n")

    # Computation of the radius of the circle of the convergent intake B.L., given the center position and two points
    hhh2 = x_start_conv + (x_start_up - x_start_conv)/200      #float(input("Inserisci la coordinata x del centro (h): "))
    kkk2 = (y_upper_conv + y_lower_conv)/2      #float(input("Inserisci la coordinata y del centro (k): "))
    xxx2 = x_start_conv     #float(input("Inserisci la coordinata x del punto sulla circonferenza: "))
    yyy2 = y_upper_conv + thick_bl    #float(input("Inserisci la coordinata y del punto sulla circonferenza: "))
    raggio2 = math.sqrt((xxx2 - hhh2)**2 + (yyy2 - kkk2)**2)
    front_nose_BL = raggio2

    file.write(f"Point({BL_conv_7}) = {{{x_start_conv + (x_start_up - x_start_conv)/200 - front_nose_BL}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Define lines for the main boundary
    file.write(f"Line({7}) = {{{boundary_point_1}, {boundary_point_6}}};\n")
    file.write(f"Line({8}) = {{{boundary_point_6}, {boundary_point_7}}};\n")
    file.write(f"Line({17}) = {{{boundary_point_7}, {new_point_end}}};\n")
    #file.write(f"Line({9}) = {{{new_point_end}, {upper_point_end}}};\n")
    file.write(f"Line({10}) = {{{upper_point_end}, {boundary_point_8}}};\n")
    file.write(f"Line({18}) = {{{boundary_point_8}, {boundary_point_2}}};\n")
    file.write(f"Line({11}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({12}) = {{{boundary_point_3}, {boundary_point_5}}};\n")
    file.write(f"Line({13}) = {{{boundary_point_5}, {boundary_point_4}}};\n")
    file.write(f"Line({14}) = {{{boundary_point_4},{1} }};\n")  # Connect back to start of bump
    file.write(f"Line({15}) = {{{new_point_start - 1},{boundary_point_1} }};\n")
    file.write(f"Line({20}) = {{{1},{boundary_point_10}}};\n")
    file.write(f"Line({21}) = {{{boundary_point_10},{boundary_point_5}}};\n")


    file.write(f"Transfinite Line{{{7}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{13}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{17}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{10}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{8}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{19}}} = {10} Using Progression {1};\n")
    #file.write(f"Transfinite Line{{{len(x_vals) + 10}}} = {150} Using Progression {1/r};\n")
    #file.write(f"Transfinite Line{{{len(x_vals) + 12}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{15}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{16}}} = {10} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{18}}} = {150} Using Progression {r};\n")

    file.write(f"Transfinite Line{{{14}}} = {300} Using Progression {1};\n")
    #file.write(f"Transfinite Line{{{19}}} = {150} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{20}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{21}}} = {300} Using Progression {1};\n")


    # Define lines for the B.L. in the convergent intake
    file.write(f"Line({30}) = {{{BL_conv_1}, {BL_conv_2}}};\n")
    file.write(f"Line({31}) = {{{BL_conv_2}, {BL_conv_3}}};\n")
    file.write(f"Circle({32}) = {{{BL_conv_3}, {circle_centre}, {BL_conv_7}}};\n")
    file.write(f"Line({33}) = {{{BL_conv_4}, {BL_conv_5}}};\n")
    file.write(f"Line({34}) = {{{BL_conv_5}, {BL_conv_6}}};\n")
    file.write(f"Line({35}) = {{{upper_point_end}, {BL_conv_1}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({36}) = {{{BL_conv_2}, {upper_point_start}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({37}) = {{{BL_conv_3}, {upper_conv}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({38}) = {{{lower_conv}, {BL_conv_4}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({39}) = {{{BL_conv_5}, {new_point_start}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({40}) = {{{BL_conv_6}, {new_point_end}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Circle({41}) = {{{BL_conv_7}, {circle_centre}, {BL_conv_4}}};\n")
    file.write(f"Line({42}) = {{{BL_conv_7}, {center_convergent}}};\n")  # vertical line in the B.L. extremis

    # Define Transfinite lines for the B.L. of the convergent intake
    file.write(f" Transfinite Line{{{3}, {4}, {30}, {34}}} = {400};\n")
    file.write(f" Transfinite Line{{{5}, {6}, {31}, {33}}} = {150};\n")
    file.write(f" Transfinite Line{{{16}, {19}, {32}, {41}}} = {50};\n")
    file.write(f" Transfinite Line{{{36}, {37}, {39}, {40}, {42}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f" Transfinite Line{{{35}, {38}}} = {n_bl} Using Progression {r};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    #file.write("Line Loop(1) = {")
    #file.write("1, 2,") # Spline 1 and 2
    #file.write(f"{15}, {7}, {8}, {17} ,{9}, {10}, {18}, {11}, {12}, {13}, {14},{19}")
    #file.write("};\n")

    file.write("Line Loop(1) = {1, 2, 15, 7, 8, 34, 33, 41, 32, 31, 30, 18, 11, 12, -21, -20};\n")

    #file.write("Line Loop(2) = {4, 6, 16, 19, 5, 3, 9};\n")

    file.write("Line Loop(2) = {14,20,21,13};\n")

    # Define 5 loops fo the B.L. of the convergent intake
    file.write("Line Loop(3) = { 35, 30, 36, -4 };\n")
    file.write("Line Loop(4) = { -36, 31, 37, -6 };\n")
    file.write("Line Loop(5) = { 37, 16, -42, -32};\n")  # on the upper circular portion (nose)
    file.write("Line Loop(6) = { 42, 19, 38, -41};\n")  # on the lower circular portion (nose)
    file.write("Line Loop(7) = { 38, 33, 39, -5 };\n")
    file.write("Line Loop(8) = { -39, 34, 40, -3 };\n")

    # Define a Line Loop for all the convergent inlet ( comprehensive of th B.L.)
    # file.write("Line Loop(10) = {34, 33, 41, 32, 31, 30, 10, 9, 17};\n")
    #file.write("Plane Surface(10) = {10,2};\n")

    #Define the fluid domain
    file.write("Plane Surface(1) = {1};\n") # It tells GMSH to mesh only on 1 and not account for 2
    file.write("Plane Surface(2) = {2};\n")
    file.write("Transfinite Surface{2} = {")
    file.write(f"{boundary_point_5},{boundary_point_4},{1},{boundary_point_10}")
    file.write("};\n")
    file.write("Recombine Surface{2};\n")

    # Define 5 surfaces for B.L. of the convergent intake
    file.write("Plane Surface(3) = {3};\n")
    file.write("Plane Surface(4) = {4};\n")
    file.write("Plane Surface(5) = {5};\n")
    file.write("Plane Surface(6) = {6};\n")
    file.write("Plane Surface(7) = {7};\n")
    file.write("Plane Surface(8) = {8};\n")
    # Define the transfinite surfaces for the B.L. of the convergent intake
    file.write("Transfinite Surface{3};\n")
    file.write("Transfinite Surface{4};\n")
    file.write("Transfinite Surface{5};\n")
    file.write("Transfinite Surface{6};\n")
    file.write("Transfinite Surface{7};\n")
    file.write("Transfinite Surface{8};\n")
    file.write("Recombine Surface{3};\n")
    file.write("Recombine Surface{4};\n")
    file.write("Recombine Surface{5};\n")
    file.write("Recombine Surface{6};\n")
    file.write("Recombine Surface{7};\n")
    file.write("Recombine Surface{8};\n")


    # Boundary Layer:

    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {1};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[1].FanPointsList = {1};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")

    file.write("Field[2] = BoundaryLayer;\n")
    file.write("Field[2].CurvesList = {2,15};\n")
    file.write("Field[2].Quads = 1;\n")
    file.write(f"Field[2].Ratio = {r};\n")
    file.write(f"Field[2].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[2].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[2].FanPointsList = {")
    file.write(f"{34}")
    file.write("};\n")
    file.write("Field[2].FanPointsSizesList = {")
    file.write(f"{5}")
    file.write("};\n")
    file.write("BoundaryLayer Field = 2;\n")


    # Physical Surface definition

    file.write("Physical Surface('VOLUME') = {1:8};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{12},{13}}};\n")

    file.write("Physical Line('lower_wall') = {")
    file.write(f"{14},")
    file.write("1, 2")
    file.write(f",{15}")
    file.write("};\n")

    #file.write(f"Physical Line('upper wall') = {{{11}}};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{3}, {4}, {6}, {16}, {19}, {5}")
    file.write("};\n")
    file.write(f"Physical Line('outlet') = {{{10}, {18},{11}}};\n")
    file.write(f"Physical Line('ENGINE') = {{{7}, {8}, {17}}};\n")



from google.colab import files
files.download("Bumpconv.geo")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Make all inlet and outlet as transfinite lines

from os import write
import numpy as np
import matplotlib.pyplot as plt
import math

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 0 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.02 # Additive quantity to the end point of the convergent to close it
thick_conv2 = 0.2 * thick_conv
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

x_start_up, y_start_up = x_vals[index_line_bump], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

x_start_conv = x_vals[index_line_bump - 1]
y_start_conv = y_start_up - (x_start - x_start_conv)*np.tan(angle_rad)


# Define the length of the line segment
End_line_param = 6
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up + line_length * np.sin(angle_rad)

# Define Far Field
Y_DIST = 50 # Multiplication factor of maximum of bump to create far field Y 150
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -5 # Multiplication factor of end x point of bump to create far field X -10
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = x_end


size_bl = 0.00001
n_bl = 65 # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1

# .geo file creation:
with open("Mesh.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.01;\n") # Size of cells close to bump and convergent
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

    # Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

    # Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")
    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1
    upper_conv = upper_point_end + 1
    lower_conv = upper_conv + 1
    circle_centre = lower_conv + 1
    center_convergent = circle_centre + 1

    # Write the convergent shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")
    file.write(f"Line({3}) = {{{new_point_start}, {new_point_end}}};\n")

    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")
    file.write(f"Line({4}) = {{{upper_point_end}, {upper_point_start}}};\n")

    y_upper_conv = y_start_conv - thick_conv2
    y_lower_conv = y_start_conv + thick_conv2 - thick_conv

    file.write(f"Point({upper_conv}) = {{{x_start_conv}, {y_upper_conv}, 0, h}};\n")
    file.write(f"Point({lower_conv}) = {{{x_start_conv}, {y_lower_conv}, 0, h}};\n")
    file.write(f"Point({circle_centre}) = {{{x_start_conv + (x_start_up - x_start_conv)/200}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Computation of the radius of the circle of the convergent intake, given the center position and two points
    hhh = x_start_conv + (x_start_up - x_start_conv)/200      #float(input("Inserisci la coordinata x del centro (h): "))
    kkk = (y_upper_conv + y_lower_conv)/2      #float(input("Inserisci la coordinata y del centro (k): "))
    xxx1 = x_start_conv     #float(input("Inserisci la coordinata x del punto sulla circonferenza: "))
    yyy1 = y_upper_conv     #float(input("Inserisci la coordinata y del punto sulla circonferenza: "))
    raggio1 = math.sqrt((xxx1 - hhh)**2 + (yyy1 - kkk)**2)
    front_nose = raggio1

    file.write(f"Point({center_convergent}) = {{{x_start_conv + (x_start_up - x_start_conv)/200 - front_nose}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Define connecting lines for closed convergent surface
    file.write(f"Line({5}) = {{{lower_conv}, {new_point_start}}};\n")
    file.write(f"Line({6}) = {{{upper_point_start}, {upper_conv}}};\n")
    file.write(f"Circle({16}) = {{{upper_conv}, {circle_centre}, {center_convergent}}};\n")  # Circle definition (start point, centre point, end point)
    file.write(f"Circle({19}) = {{{center_convergent}, {circle_centre}, {lower_conv}}};\n")  # Circle definition (start point, centre point, end point)


    # Boundary points for the entire domain
    boundary_point_1 = center_convergent + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1
    boundary_point_7 = boundary_point_6 + 1
    boundary_point_8 = boundary_point_7 + 1
    boundary_point_9 = boundary_point_8 + 1
    boundary_point_10 = boundary_point_9 + 1

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, 0, 0, H/4}};\n")

    file.write(f"Point({boundary_point_5}) = {{{X_START}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_6}) = {{{X_END}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {y_end - thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_8}) = {{{X_END}, {y_end_up + thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_10}) = {{{0}, {thick_bl}, 0, h}};\n")

    # Points for the B.L. of the convergent intake:
    BL_conv_1 = boundary_point_10 + 1
    BL_conv_2 = BL_conv_1 + 1
    BL_conv_3 = BL_conv_2 + 1
    BL_conv_4 = BL_conv_3 + 1
    BL_conv_5 = BL_conv_4 + 1
    BL_conv_6 = BL_conv_5 + 1
    BL_conv_7 = BL_conv_6 + 1

    file.write(f"Point({BL_conv_1}) = {{{x_end_up}, {y_end_up + thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_2}) = {{{x_start_up}, {y_start_up + thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_3}) = {{{x_start_conv}, {y_upper_conv + thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_4}) = {{{x_start_conv}, {y_lower_conv - thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_5}) = {{{x_start}, {y_start - thick_bl}, 0, h}};\n")
    file.write(f"Point({BL_conv_6}) = {{{x_end}, {y_end - thick_bl}, 0, h}};\n")

    # Computation of the radius of the circle of the convergent intake B.L., given the center position and two points
    hhh2 = x_start_conv + (x_start_up - x_start_conv)/200      #float(input("Inserisci la coordinata x del centro (h): "))
    kkk2 = (y_upper_conv + y_lower_conv)/2      #float(input("Inserisci la coordinata y del centro (k): "))
    xxx2 = x_start_conv     #float(input("Inserisci la coordinata x del punto sulla circonferenza: "))
    yyy2 = y_upper_conv + thick_bl    #float(input("Inserisci la coordinata y del punto sulla circonferenza: "))
    raggio2 = math.sqrt((xxx2 - hhh2)**2 + (yyy2 - kkk2)**2)
    front_nose_BL = raggio2

    file.write(f"Point({BL_conv_7}) = {{{x_start_conv + (x_start_up - x_start_conv)/200 - front_nose_BL}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Define lines for the main boundary
    file.write(f"Line({7}) = {{{boundary_point_1}, {boundary_point_6}}};\n")
    file.write(f"Line({8}) = {{{boundary_point_6}, {boundary_point_7}}};\n")
    file.write(f"Line({17}) = {{{boundary_point_7}, {new_point_end}}};\n")
    file.write(f"Line({9}) = {{{new_point_end}, {upper_point_end}}};\n")
    file.write(f"Line({10}) = {{{upper_point_end}, {boundary_point_8}}};\n")
    file.write(f"Line({18}) = {{{boundary_point_8}, {boundary_point_2}}};\n")
    file.write(f"Line({11}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({12}) = {{{boundary_point_3}, {boundary_point_5}}};\n")
    file.write(f"Line({13}) = {{{boundary_point_5}, {boundary_point_4}}};\n")
    file.write(f"Line({14}) = {{{boundary_point_4},{1} }};\n")  # Connect back to start of bump
    file.write(f"Line({15}) = {{{new_point_start - 1},{boundary_point_1} }};\n")
    file.write(f"Line({20}) = {{{1},{boundary_point_10}}};\n")
    file.write(f"Line({21}) = {{{boundary_point_10},{boundary_point_5}}};\n")


    file.write(f"Transfinite Line{{{7}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{13}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{17}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{10}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{8}}} = {50} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{19}}} = {10} Using Progression {1};\n")
    #file.write(f"Transfinite Line{{{len(x_vals) + 10}}} = {150} Using Progression {1/r};\n")
    #file.write(f"Transfinite Line{{{len(x_vals) + 12}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{15}}} = {50} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{16}}} = {10} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{18}}} = {50} Using Progression {r};\n")

    file.write(f"Transfinite Line{{{14}}} = {50} Using Progression {1};\n")
    #file.write(f"Transfinite Line{{{19}}} = {150} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{20}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{21}}} = {50} Using Progression {1};\n")


    # Define lines for the B.L. in the convergent intake
    file.write(f"Line({30}) = {{{BL_conv_1}, {BL_conv_2}}};\n")
    file.write(f"Line({31}) = {{{BL_conv_2}, {BL_conv_3}}};\n")
    file.write(f"Circle({32}) = {{{BL_conv_3}, {circle_centre}, {BL_conv_7}}};\n")
    file.write(f"Line({33}) = {{{BL_conv_4}, {BL_conv_5}}};\n")
    file.write(f"Line({34}) = {{{BL_conv_5}, {BL_conv_6}}};\n")
    file.write(f"Line({35}) = {{{upper_point_end}, {BL_conv_1}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({36}) = {{{BL_conv_2}, {upper_point_start}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({37}) = {{{BL_conv_3}, {upper_conv}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({38}) = {{{lower_conv}, {BL_conv_4}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({39}) = {{{BL_conv_5}, {new_point_start}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Line({40}) = {{{BL_conv_6}, {new_point_end}}};\n")  # vertical line in the B.L. extremis
    file.write(f"Circle({41}) = {{{BL_conv_7}, {circle_centre}, {BL_conv_4}}};\n")
    file.write(f"Line({42}) = {{{BL_conv_7}, {center_convergent}}};\n")  # vertical line in the B.L. extremis

    # Define Transfinite lines for the B.L. of the convergent intake
    file.write(f" Transfinite Line{{{3}, {4}, {30}, {34}}} = {400};\n")
    file.write(f" Transfinite Line{{{5}, {6}, {31}, {33}}} = {150};\n")
    file.write(f" Transfinite Line{{{16}, {19}, {32}, {41}}} = {50};\n")
    file.write(f" Transfinite Line{{{36}, {37}, {39}, {40}, {42}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f" Transfinite Line{{{35}, {38}}} = {n_bl} Using Progression {r};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    #file.write("Line Loop(1) = {")
    #file.write("1, 2,") # Spline 1 and 2
    #file.write(f"{15}, {7}, {8}, {17} ,{9}, {10}, {18}, {11}, {12}, {13}, {14},{19}")
    #file.write("};\n")

    file.write("Line Loop(1) = {1, 2, 15, 7, 8, 34, 33, 41, 32, 31, 30, 18, 11, 12, -21, -20};\n")

    file.write("Line Loop(2) = {4, 6, 16, 19, 5, 3, 9};\n")

    file.write("Line Loop(3) = {14,20,21,13};\n")

    # Define 5 loops fo the B.L. of the convergent intake
    file.write("Line Loop(4) = { 35, 30, 36, -4 };\n")
    file.write("Line Loop(5) = { -36, 31, 37, -6 };\n")
    file.write("Line Loop(6) = { 37, 16, -42, -32};\n")  # on the upper circular portion (nose)
    file.write("Line Loop(7) = { 42, 19, 38, -41};\n")  # on the lower circular portion (nose)
    file.write("Line Loop(8) = { 38, 33, 39, -5 };\n")
    file.write("Line Loop(9) = { -39, 34, 40, -3 };\n")

    # Define a Line Loop for all the convergent inlet ( comprehensive of th B.L.)
    file.write("Line Loop(10) = {34, 33, 41, 32, 31, 30, 10, 9, 17};\n")
    #file.write("Plane Surface(10) = {10,2};\n")

    #Define the fluid domain
    file.write("Plane Surface(1) = {1};\n")
    file.write("Plane Surface(2) = {3};\n")
    file.write("Transfinite Surface(2) = {")
    file.write(f"{boundary_point_5},{boundary_point_4},{1},{boundary_point_10}")
    file.write("};\n")
    file.write("Recombine Surface(2);\n")

    # Define 5 surfaces for B.L. of the convergent intake
    file.write("Plane Surface(4) = {4};\n")
    file.write("Plane Surface(5) = {5};\n")
    file.write("Plane Surface(6) = {6};\n")
    file.write("Plane Surface(7) = {7};\n")
    file.write("Plane Surface(8) = {8};\n")
    file.write("Plane Surface(9) = {9};\n")
    # Define the transfinite surfaces for the B.L. of the convergent intake
    file.write("Transfinite Surface{4};\n")
    file.write("Transfinite Surface{5};\n")
    file.write("Transfinite Surface{6};\n")
    file.write("Transfinite Surface{7};\n")
    file.write("Transfinite Surface{8};\n")
    file.write("Transfinite Surface{9};\n")
    file.write("Recombine Surface{4};\n")
    file.write("Recombine Surface{5};\n")
    file.write("Recombine Surface{6};\n")
    file.write("Recombine Surface{7};\n")
    file.write("Recombine Surface{8};\n")
    file.write("Recombine Surface{9};\n")


    # Physical Surface definition
    #file.write("Line Loop(20) = {14, 1, 2, 15, 7, 8, 17, -3, -5, -19, -16, -6, -4, 10, 18, 11, 12, 13};\n")
    #file.write("Plane Surface(10) = {20};\n")
    file.write("Physical Surface('VOLUME1') = {1};\n")    # main fluid field
    file.write("Physical Surface('VOLUME2') = {2};\n")     # B.L. on the flat plate
    file.write("Physical Surface('VOLUME3') = {4};\n")     # upper B.L. on the far convergent intake
    file.write("Physical Surface('VOLUME4') = {5};\n")     # upper B.L. on the starting convernging intake
    file.write("Physical Surface('VOLUME5') = {6};\n")     # upper B.L. on the circular portion of the intake
    file.write("Physical Surface('VOLUME6') = {7};\n")     # lower B.L. on the circular portion of the intake
    file.write("Physical Surface('VOLUME7') = {8};\n")     # lower B.L. on the starting convernging intake
    file.write("Physical Surface('VOLUME8') = {9};\n")     # lower B.L. on the far convergent intake


    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{12},{13} }};\n")

    file.write("Physical Line('lower_wall') = {")
    file.write(f"{14},")
    file.write("1, 2")
    file.write(f",{15}")
    file.write("};\n")

    #file.write(f"Physical Line('upper wall') = {{{11}}};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{3}, {9}, {4}, {6}, {16}, {19}, {5}")
    file.write("};\n")
    file.write(f"Physical Line('outlet') = {{{10}, {18},{11}}};\n")
    file.write(f"Physical Line('ENGINE') = {{{7}, {8}, {17}}};\n")


    # MESH:

    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {1};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[1].FanPointsList = {1};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")

    file.write("Field[2] = BoundaryLayer;\n")
    file.write("Field[2].CurvesList = {2,15};\n")
    file.write("Field[2].Quads = 1;\n")
    file.write(f"Field[2].Ratio = {r};\n")
    file.write(f"Field[2].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[2].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[2].FanPointsList = {")
    file.write(f"{34}")
    file.write("};\n")
    file.write("Field[2].FanPointsSizesList = {")
    file.write(f"{5}")
    file.write("};\n")
    file.write("BoundaryLayer Field = 2;\n")

from google.colab import files
files.download("Mesh.geo")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# CASE: DAVIDE:# CASE 1: BOUNDARY FINO A META POI NON SI INFITTISCE BENE

# Make all inlet and outlet as transfinite lines

from os import write
import numpy as np
import matplotlib.pyplot as plt

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 0 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.02 # Additive quantity to the end point of the convergent to close it
thick_conv2 = 0.45 * thick_conv
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

x_start_up, y_start_up = x_vals[index_line_bump], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

x_start_conv = x_vals[index_line_bump - 1]
y_start_conv = y_start_up - (x_start - x_start_conv)*np.tan(angle_rad)


# Define the length of the line segment
End_line_param = 4
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up + line_length * np.sin(angle_rad)

# Define Far Field
Y_DIST = 150 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -10 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = x_end


size_bl = 0.00001
n_bl = 65 # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1

# .geo file creation:
with open("Bumpconv.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.01;\n") # Size of cells close to bump and convergent
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

    # Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

    # Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")
    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1
    upper_conv = upper_point_end + 1
    lower_conv = upper_conv + 1
    circle_centre = lower_conv + 1
    center_convergent = circle_centre + 1

    # Write the convergent shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")
    file.write(f"Line({3}) = {{{new_point_start}, {new_point_end}}};\n")

    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")
    file.write(f"Line({4}) = {{{upper_point_end}, {upper_point_start}}};\n")

    file.write(f"Point({upper_conv}) = {{{x_start_conv}, {y_start_conv - thick_conv2}, 0, h}};\n")
    file.write(f"Point({lower_conv}) = {{{x_start_conv}, {y_start_conv - thick_conv2 - (0.5*thick_conv - thick_conv2)}, 0, h}};\n")
    file.write(f"Point({circle_centre}) = {{{x_start_conv + (x_start_up - x_start_conv)/200}, {y_start_conv - thick_conv2 - (0.5*thick_conv - thick_conv2)/2}, 0, h}};\n")
    file.write(f"Point({center_convergent}) = {{{x_start_conv - (x_start_up - x_start_conv)/200}, {y_start_conv - thick_conv2 - (0.5*thick_conv - thick_conv2)/2}, 0, h}};\n")

    # Define connecting lines for closed convergent surface
    file.write(f"Line({5}) = {{{lower_conv}, {new_point_start}}};\n")
    file.write(f"Line({6}) = {{{upper_point_start}, {upper_conv}}};\n")
    file.write(f"Circle({16}) = {{{upper_conv}, {circle_centre}, {lower_conv}}};\n")  # Circle definition (start point, centre point, end point)
    #file.write(f"Circle({19}) = {{{center_convergent}, {circle_centre}, {lower_conv}}};\n")  # Circle definition (start point, centre point, end point)


    # Boundary points for the entire domain
    boundary_point_1 = center_convergent + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1
    boundary_point_7 = boundary_point_6 + 1
    boundary_point_8 = boundary_point_7 + 1
    boundary_point_9 = boundary_point_8 + 1
    boundary_point_10 = boundary_point_9 + 1

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, 0, 0, H/4}};\n")

    file.write(f"Point({boundary_point_5}) = {{{X_START}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_6}) = {{{X_END}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {y_end - thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_8}) = {{{X_END}, {y_end_up + thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_9}) = {{{(X_START)/8}, {0}, 0, h}};\n")
    file.write(f"Point({boundary_point_10}) = {{{X_START/8}, {thick_bl}, 0, h}};\n")


    # Define lines for the main boundary
    file.write(f"Line({7}) = {{{boundary_point_1}, {boundary_point_6}}};\n")
    file.write(f"Line({8}) = {{{boundary_point_6}, {boundary_point_7}}};\n")
    file.write(f"Line({17}) = {{{boundary_point_7}, {new_point_end}}};\n")
    file.write(f"Line({9}) = {{{new_point_end}, {upper_point_end}}};\n")
    file.write(f"Line({10}) = {{{upper_point_end}, {boundary_point_8}}};\n")
    file.write(f"Line({18}) = {{{boundary_point_8}, {boundary_point_2}}};\n")
    file.write(f"Line({11}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({12}) = {{{boundary_point_3}, {boundary_point_5}}};\n")
    file.write(f"Line({13}) = {{{boundary_point_5}, {boundary_point_4}}};\n")
    file.write(f"Line({14}) = {{{boundary_point_4},{boundary_point_9} }};\n")  # Connect back to start of bump
    file.write(f"Line({15}) = {{{new_point_start - 1},{boundary_point_1} }};\n")
    file.write(f"Line({19}) = {{{boundary_point_9}, 1}};\n")  # Circle definition (start point, centre point, end point)
    file.write(f"Line({20}) = {{{boundary_point_9},{boundary_point_10}}};\n")
    file.write(f"Line({21}) = {{{boundary_point_10},{boundary_point_5}}};\n")


    file.write(f"Transfinite Line{{{7}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{13}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{17}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{10}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{8}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{19}}} = {150} Using Progression {r};\n")
    #file.write(f"Transfinite Line{{{len(x_vals) + 10}}} = {150} Using Progression {1/r};\n")
    #file.write(f"Transfinite Line{{{len(x_vals) + 12}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{15}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{16}}} = {10} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{18}}} = {150} Using Progression {r};\n")

    file.write(f"Transfinite Line{{{14}}} = {150} Using Progression {1};\n")
    #file.write(f"Transfinite Line{{{19}}} = {150} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{20}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{21}}} = {150} Using Progression {1};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    #file.write("Line Loop(1) = {")
    #file.write("1, 2,") # Spline 1 and 2
    #file.write(f"{15}, {7}, {8}, {17} ,{9}, {10}, {18}, {11}, {12}, {13}, {14},{19}")
    #file.write("};\n")

    file.write("Line Loop(1) = {")
    file.write("1, 2,") # Spline 1 and 2
    file.write(f"{15}, {7}, {8}, {17} ,{9}, {10}, {18}, {11}, {12}, -{21}, -{20},{19}")
    file.write("};\n")

    # inlet loop
    #file.write("Spline(20) = {")
    #file.write(f"{upper_point_start}, {upper_conv}, {center_convergent}, {lower_conv}, {new_point_start}")
    #file.write("};\n")
    #file.write("Spline(21) = {")
    #file.write(f"{center_convergent}, {lower_conv}, {new_point_start}")
    #file.write("};\n")
    file.write("Line Loop(2) = {4, 6, 16, 5, 3, 9};\n")

    file.write("Line Loop(3) = {14,20,21,13};\n")

    file.write("Plane Surface(1) = {1, 2};\n")
    file.write("Plane Surface(2) = {3};\n")
    file.write("Transfinite Surface(2) = {")
    file.write(f"{boundary_point_5},{boundary_point_4},{boundary_point_9},{boundary_point_10}")
    file.write("};\n")
    file.write("Recombine Surface(2);\n")

    # Physical Surface definition
    file.write("Physical Surface('VOLUME') = {1,2};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{12},{13} }};\n")

    file.write("Physical Line('lower wall') = {")
    file.write(f"{14},{19},")
    file.write("1, 2")
    file.write(f",{15}")
    file.write("};\n")

    #file.write(f"Physical Line('upper wall') = {{{11}}};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{3}, {9}, {4}, {6}, {16}, {5}")
    file.write("};\n")
    file.write(f"Physical Line('outlet') = {{{10}, {18},{11}}};\n")
    file.write(f"Physical Line('ENGINE') = {{{7}, {8}, {17}}};\n")


    # MESH:

    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {19,1};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write(f"Field[1].FanPointsList = {boundary_point_9};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")

    file.write("Field[2] = BoundaryLayer;\n")
    file.write("Field[2].CurvesList = {2,15};\n")
    file.write("Field[2].Quads = 1;\n")
    file.write(f"Field[2].Ratio = {r};\n")
    file.write(f"Field[2].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[2].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[2].FanPointsList = {")
    file.write(f"{34}")
    file.write("};\n")
    file.write("Field[2].FanPointsSizesList = {")
    file.write(f"{5}")
    file.write("};\n")
    file.write("BoundaryLayer Field = 2;\n")

from google.colab import files
files.download("Bumpconv.geo")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# CASE: DAVIDE:# CASE 2:

# Make all inlet and outlet as transfinite lines

from os import write
import numpy as np
import matplotlib.pyplot as plt

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 0 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.02 # Additive quantity to the end point of the convergent to close it
thick_conv2 = 0.2 * thick_conv
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

x_start_up, y_start_up = x_vals[index_line_bump], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

x_start_conv = x_vals[index_line_bump - 1]
y_start_conv = y_start_up - (x_start - x_start_conv)*np.tan(angle_rad)


# Define the length of the line segment
End_line_param = 4
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up + line_length * np.sin(angle_rad)

# Define Far Field
Y_DIST = 150 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -10 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = x_end


size_bl = 0.00001
n_bl = 65 # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1

# .geo file creation:
with open("Bumpconv.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.01;\n") # Size of cells close to bump and convergent
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

    # Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

    # Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")
    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1
    upper_conv = upper_point_end + 1
    lower_conv = upper_conv + 1
    circle_centre = lower_conv + 1
    center_convergent = circle_centre + 1

    # Write the convergent shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")
    file.write(f"Line({3}) = {{{new_point_start}, {new_point_end}}};\n")

    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")
    file.write(f"Line({4}) = {{{upper_point_end}, {upper_point_start}}};\n")

    y_upper_conv = y_start_conv - thick_conv2
    y_lower_conv = y_start_conv + thick_conv2 - thick_conv

    file.write(f"Point({upper_conv}) = {{{x_start_conv}, {y_upper_conv}, 0, h}};\n")
    file.write(f"Point({lower_conv}) = {{{x_start_conv}, {y_lower_conv}, 0, h}};\n")
    file.write(f"Point({circle_centre}) = {{{x_start_conv + (x_start_up - x_start_conv)/200}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")
    # file.write(f"Point({center_convergent}) = {{{x_start_conv - (x_start_up - x_start_conv)/200}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Define connecting lines for closed convergent surface
    file.write(f"Line({5}) = {{{lower_conv}, {new_point_start}}};\n")
    file.write(f"Line({6}) = {{{upper_point_start}, {upper_conv}}};\n")
    file.write(f"Circle({16}) = {{{upper_conv}, {circle_centre}, {lower_conv}}};\n")  # Circle definition (start point, centre point, end point)
    #file.write(f"Circle({19}) = {{{center_convergent}, {circle_centre}, {lower_conv}}};\n")  # Circle definition (start point, centre point, end point)


    # Boundary points for the entire domain
    boundary_point_1 = center_convergent + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1
    boundary_point_7 = boundary_point_6 + 1
    boundary_point_8 = boundary_point_7 + 1
    boundary_point_9 = boundary_point_8 + 1
    boundary_point_10 = boundary_point_9 + 1

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, 0, 0, H/4}};\n")

    file.write(f"Point({boundary_point_5}) = {{{X_START}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_6}) = {{{X_END}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {y_end - thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_8}) = {{{X_END}, {y_end_up + thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_10}) = {{{0}, {thick_bl}, 0, h}};\n")


    # Define lines for the main boundary
    file.write(f"Line({7}) = {{{boundary_point_1}, {boundary_point_6}}};\n")
    file.write(f"Line({8}) = {{{boundary_point_6}, {boundary_point_7}}};\n")
    file.write(f"Line({17}) = {{{boundary_point_7}, {new_point_end}}};\n")
    file.write(f"Line({9}) = {{{new_point_end}, {upper_point_end}}};\n")
    file.write(f"Line({10}) = {{{upper_point_end}, {boundary_point_8}}};\n")
    file.write(f"Line({18}) = {{{boundary_point_8}, {boundary_point_2}}};\n")
    file.write(f"Line({11}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({12}) = {{{boundary_point_3}, {boundary_point_5}}};\n")
    file.write(f"Line({13}) = {{{boundary_point_5}, {boundary_point_4}}};\n")
    file.write(f"Line({14}) = {{{boundary_point_4},{1} }};\n")  # Connect back to start of bump
    file.write(f"Line({15}) = {{{new_point_start - 1},{boundary_point_1} }};\n")
    file.write(f"Line({20}) = {{{1},{boundary_point_10}}};\n")
    file.write(f"Line({21}) = {{{boundary_point_10},{boundary_point_5}}};\n")


    file.write(f"Transfinite Line{{{7}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{13}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{17}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{10}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{8}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{19}}} = {150} Using Progression {r};\n")
    #file.write(f"Transfinite Line{{{len(x_vals) + 10}}} = {150} Using Progression {1/r};\n")
    #file.write(f"Transfinite Line{{{len(x_vals) + 12}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{15}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{16}}} = {10} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{18}}} = {150} Using Progression {r};\n")

    file.write(f"Transfinite Line{{{14}}} = {150} Using Progression {1/r};\n")
    #file.write(f"Transfinite Line{{{19}}} = {150} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{20}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{21}}} = {150} Using Progression {r};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    #file.write("Line Loop(1) = {")
    #file.write("1, 2,") # Spline 1 and 2
    #file.write(f"{15}, {7}, {8}, {17} ,{9}, {10}, {18}, {11}, {12}, {13}, {14},{19}")
    #file.write("};\n")

    file.write("Line Loop(1) = {")
    file.write("1, 2,")
    file.write(f"{15}, {7}, {8}, {17} ,{9}, {10}, {18}, {11}, {12}, -{21}, -{20}")
    file.write("};\n")

    file.write("Line Loop(2) = {4, 6, 16, 5, 3, 9};\n")

    file.write("Line Loop(3) = {14,20,21,13};\n")

    file.write("Plane Surface(1) = {1, 2};\n")
    file.write("Plane Surface(2) = {3};\n")
    file.write("Transfinite Surface(2) = {")
    file.write(f"{boundary_point_5},{boundary_point_4},{1},{boundary_point_10}")
    file.write("};\n")
    file.write("Recombine Surface(2);\n")

    # Physical Surface definition
    file.write("Physical Surface('VOLUME') = {1,2};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{12},{13} }};\n")

    file.write("Physical Line('lower wall') = {")
    file.write(f"{14},")
    file.write("1, 2")
    file.write(f",{15}")
    file.write("};\n")

    #file.write(f"Physical Line('upper wall') = {{{11}}};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{3}, {9}, {4}, {6}, {16}, {5}")
    file.write("};\n")
    file.write(f"Physical Line('outlet') = {{{10}, {18},{11}}};\n")
    file.write(f"Physical Line('ENGINE') = {{{7}, {8}, {17}}};\n")


    # MESH:

    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {1};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[1].FanPointsList = {1};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")

    file.write("Field[2] = BoundaryLayer;\n")
    file.write("Field[2].CurvesList = {2,15};\n")
    file.write("Field[2].Quads = 1;\n")
    file.write(f"Field[2].Ratio = {r};\n")
    file.write(f"Field[2].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[2].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[2].FanPointsList = {")
    file.write(f"{34}")
    file.write("};\n")
    file.write("Field[2].FanPointsSizesList = {")
    file.write(f"{5}")
    file.write("};\n")
    file.write("BoundaryLayer Field = 2;\n")

from google.colab import files
files.download("Bumpconv.geo")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# MERDA
from os import write
import numpy as np
import matplotlib.pyplot as plt

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 5 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.005 # Additive quantity to the end point of the convergent to close it
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

#x_start_up, y_start_up = x_vals[index_line_bump - Delta_conv], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

# Define the length of the line segment
End_line_param = x_vals[-1] + 4 - x_vals[-1]
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
#y_end_up = y_start_up

# Define Far Field
Y_DIST = 25 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST_START = -3 # Multiplication factor of START x point of bump to create far field X
X_DIST_END = 3 # Multiplication factor of END x point of bump to create far field X
X_START = X_DIST_START * x_vals[np.argmax(x_vals)]
X_END = X_DIST_END * x_vals[np.argmax(x_vals)]

DIAM = 0.01 # Diameter of circle at tip of convergent
y_end_up = y_start + DIAM

size_bl = 0.00001
n_bl = 65 # Number of layers  # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1

# .geo file creation:
with open("M.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.01;\n") # Size of cells close to bump
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write("h_conv = 0.001;\n") # Size of cells of Convergent
    #file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

# Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

# Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")

    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1

    # Write the convex shape points and lines 1->4->-2
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h_conv}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h_conv}};\n")
    file.write(f"Point({upper_point_start}) = {{{x_start}, {y_end_up}, 0, h_conv}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h_conv}};\n")

    file.write(f"Line({len(x_vals) + 1}) = {{{new_point_start}, {new_point_end}}};\n")
    file.write(f"Line({len(x_vals) + 2}) = {{{upper_point_start}, {upper_point_end}}};\n")
    file.write(f"Line({len(x_vals) + 4}) = {{{new_point_end}, {upper_point_end}}};\n")

    # Boundary points for the entire domain
    boundary_point_1 = upper_point_end + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1 # intermediate inlet
    boundary_point_7 = boundary_point_6 + 1 # intermediate outlet
    boundary_point_8 = boundary_point_7 + 1 # intermediate START
    boundary_point_9 = boundary_point_8 + 1 # intermediate END
    circle_centre = boundary_point_9+ 1 # Circle centre

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    #file.write(f"Point({boundary_point_1}) = {{{x_vals[-1]+0.02}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_5}) = {{{X_START}, 0, 0, H}};\n")

    file.write(f"Point({boundary_point_6}) = {{{X_START}, {thick_bl}, 0, H}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {thick_bl}, 0, H}};\n") # thick_bl

    file.write(f"Point({boundary_point_8}) = {{{0}, {thick_bl}, 0, h}};\n")
    file.write(f"Point({boundary_point_9}) = {{{x_end}, {thick_bl}, 0, h}};\n")
    #file.write(f"Point({boundary_point_9}) = {{{x_vals[-1]+0.02}, {thick_bl}, 0, h}};\n")

    # Circle definition at tip of convergent:
    file.write(f"Point({circle_centre}) = {{{(DIAM/2)+x_vals[index_line_bump]}, {(y_start + y_end_up)/2}, 0, h_conv}};\n")
    file.write(f"Circle({len(x_vals) + 20}) = {{{new_point_start}, {circle_centre}, {upper_point_start}}};\n")  # Circle definition (start point, centre point, end point)

    # Define lines for the main boundary
    #file.write(f"Line({len(x_vals) + 6}) = {{{boundary_point_1}, {boundary_point_9}}};\n") #Changed:  Connect back to END
    file.write(f"Line({len(x_vals) + 7}) = {{{boundary_point_7}, {boundary_point_3}}};\n") #changed
    file.write(f"Line({len(x_vals) + 8}) = {{{boundary_point_3}, {boundary_point_4}")
    file.write("};\n")
    file.write(f"Line({len(x_vals) + 9}) = {{{boundary_point_4}, {boundary_point_6}}};\n") # changed
    file.write(f"Line({len(x_vals) + 10}) = {{{boundary_point_5}, 1}};\n")  #Changed:  Connect back to start of bump
    file.write(f"Line({len(x_vals) + 11}) = {{{new_point_start - 1},{boundary_point_1} }};\n")

    file.write(f"Line({len(x_vals) + 12}) = {{{boundary_point_6},{boundary_point_5} }};\n")
    file.write(f"Line({len(x_vals) + 13}) = {{{boundary_point_2},{boundary_point_7} }};\n")

    #file.write(f"Line({len(x_vals) + 14}) = {{{boundary_point_5},{boundary_point_8} }};\n")
    file.write(f"Line({len(x_vals) + 15}) = {{{boundary_point_1},{boundary_point_2}}};\n")
    file.write(f"Line({len(x_vals) + 16}) = {{{1},{boundary_point_8} }};\n")
    file.write(f"Line({len(x_vals) + 17}) = {{{boundary_point_8},{boundary_point_6}}};\n")
    file.write(f"Line({len(x_vals) + 18}) = {{{boundary_point_1},{boundary_point_9}}};\n")
    file.write(f"Line({len(x_vals) + 19}) = {{{boundary_point_9},{boundary_point_7}}};\n")

    file.write(f"Transfinite Line{{{len(x_vals) + 12}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 16}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 10}}} = {150} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 17}}} = {150} Using Progression {r};\n")


    file.write(f"Transfinite Line{{{len(x_vals) + 13}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 15}}} = {150} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 18}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 19}}} = {150} Using Progression {r};\n")




    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    file.write("Line Loop(1) = {")
    file.write("1, 2") # Spline 1 and 2
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 18},{len(x_vals) + 19},{len(x_vals) + 7} ,{len(x_vals) + 8}, {len(x_vals) + 9}, -{len(x_vals) + 17},-{len(x_vals) + 16}")
    file.write("};\n")

    file.write("Line Loop(2) = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals)+20}")
    file.write("};\n")

    file.write("Line Loop(3) = {")
    file.write(f"{len(x_vals) + 12}, {len(x_vals) + 10}, {len(x_vals) + 16}, {len(x_vals) + 17}")
    file.write("};\n")

    file.write("Line Loop(4) = {")
    file.write(f"{len(x_vals) + 15}, {len(x_vals) + 13}, -{len(x_vals) + 19}, -{len(x_vals) + 18}")
    file.write("};\n")

    file.write("Plane Surface(1) = {1, 2};\n") # It tells GMSH to mesh only on 1 and not account for 2
    file.write("Plane Surface(3) = {3};\n")
    file.write("Plane Surface(4) = {4};\n")
    file.write("Transfinite Surface{3};\n")
    file.write("Transfinite Surface{4};\n")
    file.write("Recombine Surface{3};\n")
    file.write("Recombine Surface{4};\n")

    # Physical Surface definition
    file.write("Physical Surface('VOLUME') = {1};\n")
    file.write("Physical Surface('VOLUME2') = {3};\n")
    file.write("Physical Surface('VOLUME3') = {4};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{len(x_vals) + 9},{len(x_vals) + 12} }};\n")

    file.write("Physical Line('lower_wall') = {")
    file.write(f"{len(x_vals) + 10},")
    file.write("1, 2")
    file.write(f",{len(x_vals) + 11},{len(x_vals) + 15}")
    file.write("};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2},-{len(x_vals)+16}")
    file.write("};\n")

    file.write(f"Physical Line('outlet') = {{{len(x_vals) + 13},{len(x_vals) + 7}, {len(x_vals) + 8}}};\n")


    # MESH:

    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {")
    file.write("1")
    file.write("};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[1].FanPointsList = {1};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")


    file.write("Field[2] = BoundaryLayer;\n")
    file.write("Field[2].CurvesList = {")
    file.write(f"{2},{len(x_vals) + 11},{len(x_vals) + 15}")
    #file.write(f"{2}")
    file.write("};\n")
    file.write("Field[2].Quads = 1;\n")
    file.write(f"Field[2].Ratio = {r};\n")
    file.write(f"Field[2].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[2].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[2].FanPointsList = {34,55};\n")
    file.write("Field[2].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 2;\n")


from google.colab import files
files.download("M.geo")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# MERDA RIFINITA
from os import write
import numpy as np
import matplotlib.pyplot as plt

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 5 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.005 # Additive quantity to the end point of the convergent to close it
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

#x_start_up, y_start_up = x_vals[index_line_bump - Delta_conv], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

# Define the length of the line segment
End_line_param = x_vals[-1] + 4 - x_vals[-1]
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
#y_end_up = y_start_up

# Define Far Field
Y_DIST = 25 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST_START = -3 # Multiplication factor of START x point of bump to create far field X
X_DIST_END = 3 # Multiplication factor of END x point of bump to create far field X
X_START = X_DIST_START * x_vals[np.argmax(x_vals)]
X_END = X_DIST_END * x_vals[np.argmax(x_vals)]

DIAM = 0.01 # Diameter of circle at tip of convergent
y_end_up = y_start + DIAM

size_bl = 0.00001
n_bl = 65 # Number of layers  # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1

# .geo file creation:
with open("M.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.01;\n") # Size of cells close to bump
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write("h_conv = 0.001;\n") # Size of cells of Convergent
    #file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

# Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

# Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")

    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1

    # Write the convex shape points and lines 1->4->-2
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h_conv}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h_conv}};\n")
    file.write(f"Point({upper_point_start}) = {{{x_start}, {y_end_up}, 0, h_conv}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h_conv}};\n")

    file.write(f"Line({len(x_vals) + 1}) = {{{new_point_start}, {new_point_end}}};\n")
    file.write(f"Line({len(x_vals) + 2}) = {{{upper_point_start}, {upper_point_end}}};\n")
    file.write(f"Line({len(x_vals) + 4}) = {{{new_point_end}, {upper_point_end}}};\n")

    # Boundary points for the entire domain
    boundary_point_1 = upper_point_end + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1 # intermediate inlet
    boundary_point_7 = boundary_point_6 + 1 # intermediate outlet
    boundary_point_8 = boundary_point_7 + 1 # intermediate START
    boundary_point_9 = boundary_point_8 + 1 # intermediate END
    circle_centre = boundary_point_9+ 1 # Circle centre

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    #file.write(f"Point({boundary_point_1}) = {{{x_vals[-1]+0.02}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_5}) = {{{X_START}, 0, 0, H}};\n")

    file.write(f"Point({boundary_point_6}) = {{{X_START}, {thick_bl}, 0, H}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {thick_bl}, 0, H}};\n") # thick_bl

    file.write(f"Point({boundary_point_8}) = {{{0}, {thick_bl}, 0, h}};\n")
    file.write(f"Point({boundary_point_9}) = {{{x_end}, {thick_bl}, 0, h}};\n")
    #file.write(f"Point({boundary_point_9}) = {{{x_vals[-1]+0.02}, {thick_bl}, 0, h}};\n")

    # Circle definition at tip of convergent:
    file.write(f"Point({circle_centre}) = {{{(DIAM/2)+x_vals[index_line_bump]}, {(y_start + y_end_up)/2}, 0, h_conv}};\n")
    file.write(f"Circle({len(x_vals) + 20}) = {{{new_point_start}, {circle_centre}, {upper_point_start}}};\n")  # Circle definition (start point, centre point, end point)

    # Define lines for the main boundary
    #file.write(f"Line({len(x_vals) + 6}) = {{{boundary_point_1}, {boundary_point_9}}};\n") #Changed:  Connect back to END
    file.write(f"Line({len(x_vals) + 7}) = {{{boundary_point_7}, {boundary_point_3}}};\n") #changed
    file.write(f"Line({len(x_vals) + 8}) = {{{boundary_point_3}, {boundary_point_4}")
    file.write("};\n")
    file.write(f"Line({len(x_vals) + 9}) = {{{boundary_point_4}, {boundary_point_6}}};\n") # changed
    file.write(f"Line({len(x_vals) + 10}) = {{{boundary_point_5}, 1}};\n")  #Changed:  Connect back to start of bump
    file.write(f"Line({len(x_vals) + 11}) = {{{new_point_start - 1},{boundary_point_1} }};\n")

    file.write(f"Line({len(x_vals) + 12}) = {{{boundary_point_6},{boundary_point_5} }};\n")
    file.write(f"Line({len(x_vals) + 13}) = {{{boundary_point_2},{boundary_point_7} }};\n")

    #file.write(f"Line({len(x_vals) + 14}) = {{{boundary_point_5},{boundary_point_8} }};\n")
    file.write(f"Line({len(x_vals) + 15}) = {{{boundary_point_1},{boundary_point_2}}};\n")
    file.write(f"Line({len(x_vals) + 16}) = {{{1},{boundary_point_8} }};\n")
    file.write(f"Line({len(x_vals) + 17}) = {{{boundary_point_8},{boundary_point_6}}};\n")
    file.write(f"Line({len(x_vals) + 18}) = {{{boundary_point_1},{boundary_point_9}}};\n")
    file.write(f"Line({len(x_vals) + 19}) = {{{boundary_point_9},{boundary_point_7}}};\n")

    file.write(f"Transfinite Line{{{len(x_vals) + 12}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 16}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 10}}} = {150} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 17}}} = {150} Using Progression {r};\n")


    file.write(f"Transfinite Line{{{len(x_vals) + 13}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 15}}} = {150} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 18}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 19}}} = {150} Using Progression {r};\n")




    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    file.write("Line Loop(1) = {")
    file.write("1, 2") # Spline 1 and 2
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 18},{len(x_vals) + 19},{len(x_vals) + 7} ,{len(x_vals) + 8}, {len(x_vals) + 9}, -{len(x_vals) + 17},-{len(x_vals) + 16}")
    file.write("};\n")

    file.write("Line Loop(2) = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals)+20}")
    file.write("};\n")

    file.write("Line Loop(3) = {")
    file.write(f"{len(x_vals) + 12}, {len(x_vals) + 10}, {len(x_vals) + 16}, {len(x_vals) + 17}")
    file.write("};\n")

    file.write("Line Loop(4) = {")
    file.write(f"{len(x_vals) + 15}, {len(x_vals) + 13}, -{len(x_vals) + 19}, -{len(x_vals) + 18}")
    file.write("};\n")

    file.write("Plane Surface(1) = {1, 2};\n") # It tells GMSH to mesh only on 1 and not account for 2
    file.write("Plane Surface(3) = {3};\n")
    file.write("Plane Surface(4) = {4};\n")
    file.write("Transfinite Surface{3};\n")
    file.write("Transfinite Surface{4};\n")
    file.write("Recombine Surface{3};\n")
    file.write("Recombine Surface{4};\n")

    # Physical Surface definition
    file.write("Physical Surface('VOLUME') = {1};\n")
    file.write("Physical Surface('VOLUME2') = {3};\n")
    file.write("Physical Surface('VOLUME3') = {4};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{len(x_vals) + 9},{len(x_vals) + 12} }};\n")

    file.write("Physical Line('lower_wall') = {")
    file.write(f"{len(x_vals) + 10},")
    file.write("1, 2")
    file.write(f",{len(x_vals) + 11},{len(x_vals) + 15}")
    file.write("};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2},-{len(x_vals)+16}")
    file.write("};\n")

    file.write(f"Physical Line('outlet') = {{{len(x_vals) + 13},{len(x_vals) + 7}, {len(x_vals) + 8}}};\n")


    # MESH:

    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {")
    file.write("1")
    file.write("};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[1].FanPointsList = {1};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")


    file.write("Field[2] = BoundaryLayer;\n")
    file.write("Field[2].CurvesList = {")
    file.write(f"{2},{len(x_vals) + 11},{len(x_vals) + 15}")
    #file.write(f"{2}")
    file.write("};\n")
    file.write("Field[2].Quads = 1;\n")
    file.write(f"Field[2].Ratio = {r};\n")
    file.write(f"Field[2].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[2].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[2].FanPointsList = {34,55};\n")
    file.write("Field[2].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 2;\n")


from google.colab import files
files.download("M.geo")

In [ ]:
#CASE 4:
# Can be runned, all unstructured

from os import write
import numpy as np
import matplotlib.pyplot as plt

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 5 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.005 # Additive quantity to the end point of the convergent to close it
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

x_start_up, y_start_up = x_vals[index_line_bump - Delta_conv], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

# Define the length of the line segment
End_line_param = x_vals[-1] + 4 - x_vals[-1]
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up

# Define Far Field
Y_DIST = 150 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -25 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = 25 * x_vals[np.argmax(x_vals)]

# .geo file creation:
with open("geometry.geo", "w") as file:

    file.write("h = 0.01;\n") # Size of cells close to bump and convergent
    file.write("H = 1.5;\n") # Size of cells close to farfield
    file.write("r = 1.0;\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

# Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

# Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")

    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1

    # Write the convex shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")
    file.write(f"Line({len(x_vals) + 1}) = {{{new_point_start}, {new_point_end}}};\n")

    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")
    file.write(f"Line({len(x_vals) + 2}) = {{{upper_point_start}, {upper_point_end}}};\n")

    # Define connecting lines for closed convex surface
    file.write(f"Line({len(x_vals) + 3}) = {{{new_point_start}, {upper_point_start}}};\n")
    file.write(f"Line({len(x_vals) + 4}) = {{{new_point_end}, {upper_point_end}}};\n")

    # Boundary points for the entire domain
    boundary_point_1 = upper_point_end + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_5}) = {{{X_START}, 0, 0, H}};\n")

    # Define lines for the main boundary
    file.write(f"Line({len(x_vals) + 6}) = {{{boundary_point_1}, {boundary_point_2}}};\n")
    file.write(f"Line({len(x_vals) + 7}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({len(x_vals) + 8}) = {{{boundary_point_3}, {boundary_point_4}")
    file.write("};\n")
    file.write(f"Line({len(x_vals) + 9}) = {{{boundary_point_4}, {boundary_point_5}}};\n")
    file.write(f"Line({len(x_vals) + 10}) = {{{boundary_point_5}, 1}};\n")  # Connect back to start of bump
    file.write(f"Line({len(x_vals) + 11}) = {{{new_point_start - 1},{boundary_point_1} }};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    file.write("Line Loop(1) = {")
    file.write("1, 2") # Spline 1 and 2
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6}, {len(x_vals) + 7}, {len(x_vals) + 8}, {len(x_vals) + 9}, {len(x_vals) + 10}")
    file.write("};\n")

    file.write("Line Loop(2) = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals) + 3}")
    file.write("};\n")

    file.write("Plane Surface(1) = {1, 2};\n") # It tells GMSH to mesh only on 1 and not account for 2

    # Physical Surface definition
    file.write("Physical Surface('VOLUME') = {1};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{len(x_vals) + 9}}};\n")

    file.write("Physical Line('lower wall') = {")
    file.write(f"{len(x_vals) + 10},")
    file.write("1, 2") # Spline 1 and 2
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6}")
    file.write("};\n")

    file.write(f"Physical Line('upper wall') = {{{len(x_vals) + 8}}};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals) + 3}")
    file.write("};\n")

    file.write(f"Physical Line('outlet') = {{{len(x_vals) + 7}}};\n")


    # MESH:

# 1: MeshAdapt, 2: Automatic, 3: Initial mesh only, 5: Delaunay, 6: Frontal-Delaunay,
# 7: BAMG, 8: Frontal-Delaunay for Quads, 9: Packing of Parallelograms

    file.write("Mesh.Algorithm = 6; // Frontal-Delaunay for quads\n")
    file.write("Mesh.RecombineAll = 1; // Enable recombination for quad elements\n")

    # Download the file (if running in an environment that supports file download)
from google.colab import files
files.download("geometry.geo")